In [2]:
###########################################################################################################
# CALIBRATION FACTOR COLLECTION
# 
# A new calibration factor must be collected if:  
#      - you are collecting data on a new day
#      - you have realigned anty optical components 
#      - you have rebalanced the photodiode
#      - you have changed any optical components
#      - you have changed temperatures
#      - you have changed cells
#      - you have changed laser wavelength or power
###########################################################################################################


from ScopeChat import Oscope
import CalibrationFactor as cf
import Utilities as util

#record experiment variables 
optical_len = 2.00 #optical length, in cm
oven_temp = 120 #oven temp, in Celcius
laser_power = 0.400 #laser power on readout
laser_temp = 1.000 #laser temp on readout
laser_wavelen = 7.80723E-5 #laser wavelength, in cm
lockin_sensitivity = 0.02 #in VOLTS. THIS IS IMPORTANT
physical_rotation = 1.0 #the angle (in degrees) rotated on the dial. Note that 1 degree = 4 rotations of the small dial
calibration_rotation = 2 * physical_rotation #angle used to obtain calibration

#set up scope communication
#same as the actual data collection here
visa_address = 'USB0::0x0699::0x0368::C041014::INSTR'
source_channel = 'CH1'
my_scope = Oscope(visa_address, 16, source_channel)
my_scope.startScope()
my_scope.collectionSetUp()

#save the two collected voltages and their error values to an array
cal_volts = my_scope.collectCalibrationMeasurement()

#close the connection to the scope
my_scope.scopeClose()

#convert the voltage difference between the two measurements into a number with units Radians/Volt
cal_info = cf.CalibrationFactor(calibration_rotation, lockin_sensitivity, cal_volts[0], cal_volts[1], cal_volts[2], cal_volts[3], laser_wavelen, optical_len, oven_temp, laser_power, laser_temp)
#format the collected and calculated calibration info to be saved to a JSON file
cal_info_conv = cal_info.calFPrepforJSON()

#now save calibration factors to their own file
ds = util.getDateString()
filename = 'Data/calibration_'+ds+'.json'
util.saveDatatoJSON(filename, cal_info_conv)

[-10.729761981965, 0.003041505824999735]
[6.639173603064999, 0.0036014318500001696]


In [6]:
###########################################################################################################
# DATA COLLECTION
# 
# Note that this does not include calibration any more. If you have not done a calibration for your current 
#  set up or you a beginning a new set of trials, take a calibration measurement first
# 
# IMPORTANT: CURRENTLY YOU WILL NEED TO MANUALLY COPY THE CALIBRATION FACTOR FROM THE CALIBRATION
#    FACTOR FILE INTO THIS MODULE. DO NOT FORGET TO DO THIS BEFORE COLLECTING DATA
#
# IMPORTANT: YOU MUST INCLUDE THE LOCKIN SENSITIVITY SETTINGS YOU ARE USING TO COLLECT DATA. THIS IS 
#    NEEDED TO PROPERLY CONVERT FROM VOLTAGE ON THE SCOPE TO ROTATION ANGLE.
###########################################################################################################
from ScopeChat import Oscope
import densityCalculations as calc
import Utilities as util

#record experiment variables 
trial = 16
olen = 2.00 #optical length
t = 120 #oven temp
power = 0.400 #laser power on readout
laserTemp = 1.000 #laser temp on readout
laserwl = 7.80723E-5 #laser wavelength
#record and equipment settings and calibration factors
lockin_sensitivity = 0.002 #IN VOLTS
lockin_scaling_factor = lockin_sensitivity / 10.0 #this is necessary to ensure that voltage is converted to rotation correctly
time_const = 1 #set timeconstant used on lockin
cal_factor = 1.0048567705545528 #set calibration factor to use for dataset
conversion_f = cal_factor * lockin_scaling_factor #combines conversion factor and data collection settings

#set up scope communication
visa_address = 'USB0::0x0699::0x0368::C041014::INSTR'
source_channel = 'CH1'
my_scope = Oscope(visa_address, 16, source_channel)
my_scope.startScope()
my_scope.collectionSetUp()
#collect data 
raw_data = my_scope.collectUntilDone(time_const)
#close the channel to the scope once done collecting data
my_scope.scopeClose()

#save the raw data to a file so I have if as backup
filename_raw = 'Data/rawdata.json'
raw = util.prepRawDataForFile(raw_data)
util.saveDatatoJSON(filename_raw, raw)

#now convert the raw data into analysis and save that to a separate file
ds = util.getDateString()
filename_converted = 'Data/rotationdata_'+ds+'.json'
converted_data = calc.convertRawDataArray(raw_data, conversion_f)
converted = util.prepConvertedDataforFile(converted_data, trial, laserwl, olen, t,power,conversion_f,laserTemp)
util.saveDatatoJSON(filename_converted, converted)

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [2]:
###########################################################################################################
# Continuous Voltage collection
# 
# This module collects voltage data continuously at specified intervals until prompted to stop
###########################################################################################################

from ScopeChat import Oscope
import numpy as np
import threading
from DataPoint import VoltageReading
import time
import datetime
import Utilities as util

#TODO : udpate this program to use the single value collection method instead

computer = util.whatSystemIsthis()

#set up scope communication
visa_address = 'USB0::0x0699::0x0368::C041014::INSTR' #visa address for scope
source_channel = 'CH1' #channel to read from
my_scope = Oscope(visa_address, 16, source_channel) #open communication with scope
voltage_data = np.array([]) #stores output of measurements
flag = 1 #used to trigger threading behavior
N = 1 #number of voltage values to collect and avgerage per data point. NOTE: If N=1 uncertainty is listed as 0.0

#this function represents one of the two threads needed to continuously collect
#this function will collected mean values approximately once per .5s from the oscilloscope
def collectData():
    global flag
    global voltage_data
    my_scope.startScope()
    my_scope.collectionSetUp()
    #print('Starting Collection.')
    while flag==1:
        val = my_scope.collectVoltage()
        #val = my_scope.collectMean(N)
        val2 = VoltageReading(val[0], val[1])
        voltage_data = np.append(voltage_data, val2)
        time.sleep(0.5)
        if flag==False:
            my_scope.scopeClose()


#this function represents one of the two threads needed to continuously collect
#this function waits for the user to enter something (anything) on the keyboard. 
#once something is entered collection will stop. 
def get_input():
    global flag
    keystrk=input('Press any key to end collection \n')
    # thread doesn't continue until key is pressed
    flag=False
    time.sleep(1) #allow the threads to catch up to eachother
    print('Ending data collection.')

dt = datetime.datetime.today()
dt = str(dt).replace(" ", "_") #on windows comment out this line and uncomment line below
#dt = dt.replace(':', '-') #only use on windows

print('Starting Collection.')
#threading!
#I hate it.
threads = []  # list to hold threads if you have more than one
col=threading.Thread(target=collectData)
threads.append(col)
inp=threading.Thread(target=get_input)
threads.append(inp)
col.start()
inp.start()
for thread in threads:  # wait for all threads to finish
    thread.join() 

#Code below this will only execute once the threads complete

#format data to be saved to csv file.
fields = ['Voltage', 'Collection Time']
formatted_data = []
for i in voltage_data:
    row = [i.voltage, i.timestamp]
    formatted_data.append(row)

#construct file name
#TODO: select the correct file naming convention base on operating system
# will add this later because it's annoying to do and not super high priority right now
if (computer == 'Windows'):
    filename = 'Data\\VoltageReadouts\\'+dt+'.csv' #windows file naming convention
else: 
    filename = 'Data/VoltageReadouts/'+dt+'.csv' #mac file naming convention


#save to file
#uses date and time to create unique file names
util.exportToCSV(filename, fields, formatted_data)

Starting Collection.
Ending data collection.


OSError: [Errno 22] Invalid argument: 'Data\\VoltageReadouts\\2024-06-25_11:43:19.757814.csv'